# Latent-Space Counterfactual Explanations for Heart Disease Dataset

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import torch
from torch import nn
import torch.nn.functional as F

In [ ]:
df = pd.read_csv('/mnt/data/Cleaned_heart_data3.csv')

X = df.drop('target', axis=1).values
y = df['target'].values

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)

input_dim = X.shape[1]

In [ ]:
class Classifier(nn.Module):
    def __init__(self, input_dim, hidden=64):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden), nn.ReLU(),
            nn.Linear(hidden, hidden), nn.ReLU(),
            nn.Linear(hidden, 2)
        )
    def forward(self, x): return self.net(x)

clf = Classifier(input_dim)
opt_clf = torch.optim.Adam(clf.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()

for epoch in range(40):
    clf.train()
    opt_clf.zero_grad()
    pred = clf(X_train)
    loss = loss_fn(pred, y_train)
    loss.backward()
    opt_clf.step()

In [ ]:
class VAE(nn.Module):
    def __init__(self, input_dim, latent_dim=8):
        super().__init__()
        self.encoder = nn.Sequential(nn.Linear(input_dim,32), nn.ReLU(), nn.Linear(32, latent_dim*2))
        self.decoder = nn.Sequential(nn.Linear(latent_dim,32), nn.ReLU(), nn.Linear(32,input_dim))

    def encode(self,x):
        h=self.encoder(x)
        return h[:,:h.shape[1]//2], h[:,h.shape[1]//2:]

    def sample(self,mu,logvar):
        std=torch.exp(0.5*logvar)
        return mu + torch.randn_like(std)*std

    def decode(self,z): return self.decoder(z)

    def forward(self,x):
        mu,logvar=self.encode(x)
        z=self.sample(mu,logvar)
        return self.decode(z),mu,logvar

vae = VAE(input_dim)
opt_vae = torch.optim.Adam(vae.parameters(), lr=1e-3)

def vae_loss(x,recon,mu,logvar):
    recon_loss=((x-recon)**2).mean()
    kl = -0.5*torch.mean(1+logvar - mu.pow(2) - logvar.exp())
    return recon_loss + 0.01*kl

for epoch in range(60):
    opt_vae.zero_grad()
    recon,mu,logvar=vae(X_train)
    loss=vae_loss(X_train,recon,mu,logvar)
    loss.backward()
    opt_vae.step()

In [ ]:
def generate_cf(x, target_label, steps=600, lr=0.05, lam=1.0):
    clf.eval(); vae.eval()
    mu, logvar = vae.encode(x)
    z = mu.clone().detach(); z.requires_grad=True
    opt = torch.optim.Adam([z], lr=lr)

    for _ in range(steps):
        opt.zero_grad()
        x_cf = vae.decode(z)
        pred = clf(x_cf)
        ce = loss_fn(pred, torch.tensor([target_label]))
        dist = torch.norm(z-mu)
        loss = lam*ce + dist
        loss.backward(); opt.step()
    return vae.decode(z).detach()

In [ ]:
def evaluate_cf(x, cf):
    x_np, cf_np = x.numpy().flatten(), cf.numpy().flatten()
    return {
        'proximity': np.linalg.norm(x_np-cf_np),
        'sparsity': np.sum(x_np!=cf_np),
        'realism_mse': np.mean((x_np-cf_np)**2)
    }

i=5
x=X_test[i].unsqueeze(0)
pred=torch.argmax(clf(x)).item()
target_label=1-pred
cf=generate_cf(x,target_label)

original=scaler.inverse_transform(x.numpy())
counter=scaler.inverse_transform(cf.numpy())
metrics=evaluate_cf(x,cf)

original, counter, metrics